In [1]:
import tensorflow as tf
import jpeg_related_functions as jrf
import ML_ADMM_2d as mlcsc
import numpy as np
import pickle as pkl
import datetime
import util

In [2]:
rho = 1.
alpha_init = 1.5
mu_init = 1.
b_init = 0.
lraParam = {'n_components': 4}
cmplxdtype = tf.complex128 # This should really be elsewhere.
batch_size = 1
noe_per_save = 1
num_of_saves = 2
step_size = 0.1

In [3]:
databasename = 'simpleTest/'
experimentpath = 'data/experiment/' + databasename + 'experiment1/'
def checkpointfilename(ii):
    return 'checkpoint_epoch' + str(ii) + '.ckpt'
modelfilename = 'initial_model.ckpt'
fid = open(experimentpath + 'problem_param.pckl','rb')
problem_param = pkl.load(fid)
fid.close()
data_param = problem_param['data_param']
#targetSz = data_param['target_size']
targetSz = (32,32)
qY = data_param['qY']
qUV = data_param['qUV']
strides = problem_param['stride']
fltrSz = problem_param['fltrSz']
real_dtype = data_param['dtype']
noi = problem_param['noi']
noL = problem_param['noL']
noc = problem_param['noc']
datapath = problem_param['datapath']
trainfile = problem_param['trainfile']
#padding = data_param['padding']
padding = ((13,13),(13,13))


In [4]:
cropAndMerge = mlcsc.CropPadObject(targetSz,strides,[np.asarray(ks) for ks in fltrSz],real_dtype)
paddingTuple = cropAndMerge.paddingTuple
fftSz = cropAndMerge.get_fft_size(targetSz,strides)
paddingDiff = ((padding[0][0] - paddingTuple[0][0],padding[0][1] - paddingTuple[0][1]),(padding[1][0] - paddingTuple[1][0],padding[1][1] - paddingTuple[1][1]))
assert(paddingDiff[0][0] >= 0)
assert(paddingDiff[0][1] >= 0)
assert(paddingDiff[1][0] >= 0)
assert(paddingDiff[1][1] >= 0)
print(paddingDiff)


((8, 7), (8, 7))


In [5]:
startr = paddingDiff[0][0]
startc = paddingDiff[1][0]
endr = targetSz[0] + padding[0][0] + padding[0][1] - paddingDiff[0][1]
endc = targetSz[1] + padding[1][0] + padding[1][1] - paddingDiff[1][1]
example_structure = {'highpass': tf.io.FixedLenFeature([], tf.string), 'lowpass': tf.io.FixedLenFeature([], tf.string), 'compressed': tf.io.FixedLenFeature([], tf.string),'raw': tf.io.FixedLenFeature([], tf.string)}

def restore_double(x):
    return tf.io.parse_tensor(x,real_dtype)

def _parse_image_function(example_proto):
    x = tf.io.parse_single_example(example_proto, example_structure)
    highpass = restore_double(x['highpass'])
    lowpass = restore_double(x['lowpass'])
    return ((highpass[slice(startr,endr),slice(startc,endc),slice(None)],lowpass[slice(startr,endr),slice(startc,endc),slice(None)],restore_double(x['compressed'])),restore_double(x['raw']))

raw_dataset = tf.data.TFRecordDataset([datapath + trainfile])
dataset = raw_dataset.map(_parse_image_function)
dataset_batch = dataset.batch(batch_size)


In [6]:
for (x,y) in dataset_batch:
    print(x[0].shape)
    print(x[1].shape)
    print(x[2].shape)
    print(y.shape)
    print(tf.reduce_max(tf.abs(cropAndMerge.crop(x[0]) + cropAndMerge.crop(x[1]) - x[2])))
    croppedHighpass = x[0][slice(None),slice(paddingTuple[0][0], paddingTuple[0][0] + targetSz[0]),slice(paddingTuple[1][0],paddingTuple[1][0] + targetSz[1]),slice(None)]
    croppedLowpass = x[1][slice(None),slice(paddingTuple[0][0],paddingTuple[0][0] + targetSz[0]),slice(paddingTuple[1][0],paddingTuple[1][0] + targetSz[1]),slice(None)]
    print(croppedHighpass.shape)
    print(croppedLowpass.shape)
    print(x[2].shape)
    print(tf.reduce_max(tf.abs(croppedHighpass + croppedLowpass - x[2])))
    print(tf.reduce_max(x[2]))
    print(tf.reduce_min(x[2]))
    break

(1, 43, 43, 1)
(1, 43, 43, 1)
(1, 32, 32, 1)
(1, 32, 32, 3)
tf.Tensor(0.0, shape=(), dtype=float64)
(1, 32, 32, 1)
(1, 32, 32, 1)
(1, 32, 32, 1)
tf.Tensor(0.0, shape=(), dtype=float64)
tf.Tensor(0.8356980392156863, shape=(), dtype=float64)
tf.Tensor(0.3130117647058823, shape=(), dtype=float64)


In [7]:
#   ******** BUILD MODEL ********
CSC = mlcsc.MultiLayerCSC(rho,alpha_init,mu_init,b_init,cropAndMerge,fftSz,strides,problem_param['D'],lraParam,noi,noL,cmplxdtype)

In [8]:
highpassShape = (targetSz[0] + paddingTuple[0][0] + paddingTuple[0][1],targetSz[1] + paddingTuple[1][0] + paddingTuple[1][1],noc)
highpass = tf.keras.Input(shape=highpassShape,dtype=real_dtype)
lowpass = tf.keras.Input(shape = highpassShape,dtype = real_dtype)
compressed = tf.keras.Input(shape = (targetSz[0],targetSz[1],noc),dtype= real_dtype)
inputs = (highpass,lowpass,compressed)

reconstruction,itstats = CSC(inputs)
#rgb_reconstruction = jrf.YUV2RGB(dtype=real_dtype)(reconstruction)
#clipped_reconstruction = util.clip(a = 0.,b = 1.,dtype=real_dtype)(rgb_reconstruction)
import post_process_grad as ppg
model = ppg.Model_PostProcess(inputs,reconstruction)

xprevlayer_shape:  (None, 43, 43, 1, 1)
D_shape:  (1, 11, 11, 1, 64)
xcurrlayer_shape:  (None, 43, 43, 64, 1)
still xcurrlayer_shape:  (None, 43, 43, 64, 1)
z_over_R shape:  (None, 43, 43, 64, 1)
z_prevlayer_shape:  (None, 43, 43, 1, 1)
z_over_R_shape:  (None, 43, 43, 64, 1)
gamma_scaled:  (None, 43, 43, 64, 1)
z_over_R shape:  (None, 43, 43, 64, 1)
x_0_shape:  (None, 43, 43, 64, 1)
Dx_shape:  (None, 43, 43, 1, 1)
z_over_R shape:  (None, 43, 43, 64, 1)
z_prevlayer_shape:  (None, 43, 43, 1, 1)
z_over_R_shape:  (None, 43, 43, 64, 1)
gamma_scaled:  (None, 43, 43, 64, 1)
z_over_R shape:  (None, 43, 43, 64, 1)
x_0_shape:  (None, 43, 43, 64, 1)
Dx_shape:  (None, 43, 43, 1, 1)
z_over_R shape:  (None, 43, 43, 64, 1)
z_prevlayer_shape:  (None, 43, 43, 1, 1)
z_over_R_shape:  (None, 43, 43, 64, 1)
gamma_scaled:  (None, 43, 43, 64, 1)
z_over_R shape:  (None, 43, 43, 64, 1)
x_0_shape:  (None, 43, 43, 64, 1)
Dx_shape:  (None, 43, 43, 1, 1)
z_over_R shape:  (None, 43, 43, 64, 1)
z_prevlayer_shape:  (

z_over_R shape:  (None, 43, 43, 64, 1)
z_prevlayer_shape:  (None, 43, 43, 1, 1)
z_over_R_shape:  (None, 43, 43, 64, 1)
gamma_scaled:  (None, 43, 43, 64, 1)


In [8]:
for (x,y) in dataset_batch:
    x_input = x
    x_output = y
    break



In [9]:
s = CSC.preprocess(x_input)
#y,u,By,negC,itstats = CSC.init_vars(s)


In [10]:
negC = CSC.get_negative_C(s)
temp,s_crop = negC
print('negC_shape: ',s_crop.shape)
x,Ax = CSC.init_x(s,negC)
s_HP,s_LP,cmprssd = s
print('s_HP_shape: ',s_HP.shape)
print('x[0]_shape: ',x[0].shape)
print('Ax_layer[0]_shape: ',Ax[1][0].shape)
y,By = CSC.init_y(s,x,Ax,negC)
v,z = y
print('v_shape: ',v.shape)
print('z[0]_shape',z[0].shape)
u = CSC.init_u(s,Ax,By,negC)
eta,gamma = u
print('eta_shape: ',eta.shape)
print('gamma[0]_shape: ',gamma[0].shape)
itstats = CSC.init_itstats(s)

negC_shape:  (1, 32, 32, 1)
xprevlayer_shape:  (1, 43, 43, 1, 1)
D_shape:  (1, 11, 11, 1, 64)
xcurrlayer_shape:  (1, 43, 43, 64, 1)
still xcurrlayer_shape:  (1, 43, 43, 64, 1)
s_HP_shape:  (1, 43, 43, 1)
x[0]_shape:  (1, 43, 43, 64, 1)
Ax_layer[0]_shape:  (1, 43, 43, 64, 1)
v_shape:  (1, 43, 43, 1, 1)
z[0]_shape (1, 43, 43, 64, 1)
z_over_R shape:  (1, 43, 43, 64, 1)
eta_shape:  (1, 32, 32, 1)
gamma[0]_shape:  (1, 43, 43, 64, 1)


In [11]:
v,z = y
eta,gamma = u
Bzero,Bz = By


Currently, eta is the only variable with the correct shape.  Need to figure out what went wrong in initialization.

In [12]:
print('v_shape: ',v.shape)
print('z[0]_shape: ',z[0].shape)
#print('z[1].shape: ',z[1].shape)
print('eta_shape: ',eta.shape)
print('Bz[0].shape: ',Bz[0].shape)
#print('Bz[1].shape: ',Bz[1].shape)

v_shape:  (1, 43, 43, 1, 1)
z[0]_shape:  (1, 43, 43, 64, 1)
eta_shape:  (1, 32, 32, 1)
Bz[0].shape:  (1, 43, 43, 64, 1)


In [13]:
x,Ax = CSC.xstep(y,u,By,negC)
print('x[0]_shape: ',x[0].shape)
temp,Ax_layers = Ax
print('Ax_layers[0]_shape: ',Ax_layers[0].shape)
#uhalf = CSC.relax(u,Ax,By,negC)

z_prevlayer_shape:  (1, 43, 43, 1, 1)
z_over_R_shape:  (1, 43, 43, 64, 1)
gamma_scaled:  (1, 43, 43, 64, 1)
x[0]_shape:  (1, 43, 43, 64, 1)
Ax_layers[0]_shape:  (1, 43, 43, 64, 1)


In [14]:
uhalf = CSC.relax(u,Ax,By,negC)

z_over_R shape:  (1, 43, 43, 64, 1)


In [21]:
rho = CSC.rho
#if CSC.strides[0] == 2:
#    mu = CSC.updateZ[0][0].mu
#else:
mu = CSC.updateZ_lastlayer.mu
Azero,Ax = Ax_relaxed
v,z = y
Bv,Bz = By
eta,gamma = u
Dx = CSC.IFFT[0](CSC.dictObj[0].dmul(x[0]))
augLangbefore = mu/2*CSC.reconstructionTerm(CSC.IFFT[0](v),Dx) + CSC.jpegConstraint_relaxed(eta,Azero,Bv)

In [15]:
y,By = CSC.ystep(x,uhalf,Ax,negC)

x_0_shape:  (1, 43, 43, 64, 1)
Dx_shape:  (1, 43, 43, 1, 1)


In [16]:
v,z = y
print('v_shape: ',v.shape)
print('z[0]_shape: ',z[0].shape)
Bzero,Bz = By
print('Bzero: ',Bzero.shape)
print('Bz[0]_shape: ',Bz[0].shape)

v_shape:  (1, 43, 43, 1, 1)
z[0]_shape:  (1, 43, 43, 64, 1)
Bzero:  (1, 32, 32, 1)
Bz[0]_shape:  (1, 43, 43, 64, 1)


In [17]:
u = CSC.ustep(uhalf,Ax,By,negC)

z_over_R shape:  (1, 43, 43, 64, 1)


In [18]:
eta,gamma = u
print('eta: ',eta.shape)
print('gamma[0]: ',gamma[0].shape)

eta:  (1, 32, 32, 1)
gamma[0]:  (1, 43, 43, 64, 1)


In [31]:
x,Ax = CSC.xstep(y,u,By,negC)

z_prevlayer_shape:  (1, 43, 43, 1, 1)
z_over_R_shape:  (1, 43, 43, 64, 1)
gamma_scaled:  (1, 43, 43, 64, 1)


In [32]:
uhalf = CSC.relax(u,Ax,By,negC)

z_over_R shape:  (1, 43, 43, 64, 1)


In [33]:
y,By = CSC.ystep(x,uhalf,Ax,negC)

x_0_shape:  (1, 43, 43, 64, 1)
Dx_shape:  (1, 43, 43, 1, 1)


In [34]:
u = CSC.ustep(uhalf,Ax,By,negC)

z_over_R shape:  (1, 43, 43, 64, 1)


In [23]:
v,z = y
Bv,Bz = By

augLangafter = mu/2*CSC.reconstructionTerm(CSC.IFFT[0](v),Dx) + CSC.jpegConstraint_relaxed(eta,Azero,Bv)

print(augLangbefore - augLangafter)

tf.Tensor(39227.8656928235, shape=(), dtype=float64)


In [24]:
u = CSC.ustep(u,Ax_relaxed,By,negC)

In [10]:
s = CSC.preprocess(x_input)
y,u,By,negC,itstats = CSC.init_vars(s)


In [11]:
x,Ax = CSC.xstep(y,u,By,negC)
Ax_relaxed = CSC.relax(Ax,By,negC)

In [12]:
rho = CSC.rho
#if CSC.strides[0] == 2:
#    mu = CSC.updateZ[0][0].mu
#else:
mu = CSC.updateZ_lastlayer.mu
Azero,Ax = Ax_relaxed
v,z = y
Bv,Bz = By
eta,gamma = u
Dx = CSC.IFFT[0](CSC.dictObj[0].dmul(x[0]))
augLangbefore = mu/2*CSC.reconstructionTerm(CSC.IFFT[0](v),Dx) + CSC.jpegConstraint_relaxed(eta,Azero,Bv)

In [13]:
import util
vplus_LF,Bv = CSC.updatev((CSC.cropAndMerge.crop(tf.squeeze(Dx,axis=-1)) + CSC.cropAndMerge.crop(x_input[1]),Azero,eta))
v_small = vplus_LF - CSC.cropAndMerge.crop(x_input[1])
v = CSC.cropAndMerge.merge((v_small,tf.squeeze(Dx,axis=-1)))
augLangafter = mu/2*CSC.reconstructionTerm(v,tf.squeeze(Dx,axis=-1)) + CSC.jpegConstraint_relaxed(eta,Azero,Bv)
print(augLangbefore - augLangafter)
y = (CSC.FFT[0](util.addDim(v)),Bv)

tf.Tensor(161593.26288551564, shape=(), dtype=float64)


In [ ]:
u = CSC.ustep(u,Ax_relaxed,By,negC)

In [23]:
print(rho)
print(CSC.updatev.rho)
print(CSC.updatev.mu)
print(mu)

tf.Tensor(1.0, shape=(), dtype=float64)
tf.Tensor(1.0, shape=(), dtype=float64)
<tf.Variable 'Variable:0' shape=() dtype=float64, numpy=1.0>
<tf.Variable 'Variable:0' shape=() dtype=float64, numpy=1.0>


In [10]:
s = CSC.preprocess(x_input)
y,u,By,negC,itstats = CSC.init_vars(s)


In [15]:
x,Ax = CSC.xstep(y,u,By,negC)

In [16]:
Ax_relaxed = CSC.relax(Ax,By,negC)

In [17]:
beforeyupdate = CSC.evaluateRelaxedLagrangian(x,y,u,Ax_relaxed,By,negC)
#beforeyupdate = CSC.evaluateLagrangian(x,y,u,By,negC)
y,By = CSC.ystep(x,u,Ax_relaxed,negC)
#afteryupdate = CSC.evaluateLagrangian(x,y,u,By,negC)
afteryupdate = CSC.evaluateRelaxedLagrangian(x,y,u,Ax_relaxed,By,negC)

print('y-step improvement: ',beforeyupdate - afteryupdate)

y-step improvement:  tf.Tensor(9243.85075034016, shape=(), dtype=float64)


In [18]:
u = CSC.ustep(u,Ax_relaxed,By,negC)

In [19]:
beforexupdate = CSC.evaluateLagrangian(x,y,u,By,negC)
x,Ax = CSC.xstep(y,u,By,negC)
afterxupdate = CSC.evaluateLagrangian(x,y,u,By,negC)

print('x-step improvement: ',beforexupdate - afterxupdate)

x-step improvement:  tf.Tensor(10208.252650545322, shape=(), dtype=float64)


In [21]:
print(tf.reduce_max(x_input[2] - CSC.cropAndMerge.crop(CSC.cropAndMerge.merge((x_input[2],x_input[0])))))

tf.Tensor(0.0, shape=(), dtype=float64)


In [22]:
print(tf.reduce_max(x_input[0] - CSC.cropAndMerge.merge((x_input[2],x_input[0]))))

tf.Tensor(0.03007777523421025, shape=(), dtype=float64)


In [23]:
print(tf.reduce_max(x_input[1] - CSC.cropAndMerge.merge((x_input[2],x_input[1]))))

tf.Tensor(0.3684579105100836, shape=(), dtype=float64)


In [26]:
print(tf.reduce_max(CSC.cropAndMerge.crop(x_input[0]) + CSC.cropAndMerge.crop(x_input[1]) - x_input[2]))

tf.Tensor(2.7755575615628914e-17, shape=(), dtype=float64)
